In [15]:
from collections import defaultdict as ddict
from typing import DefaultDict, Set
import itertools

import freeman as fm

import pandas as pd
import numpy as np
import networkx as nx
import statsmodels.api as sm

In [2]:
teg = pd.read_csv(
    "csv/results.csv",
     usecols=("team_1", "team_2", "match_id", "event_id", "date"),
)
bruno = pd.read_csv(
    "csv/players.csv",
     usecols=("date", "team", "player_id", "match_id", "event_id", "rating"),
)

In [3]:
cutoff_date = "2017-06-01"
if cutoff_date:
    bruno = bruno[bruno["date"] >= cutoff_date]
    teg = teg[teg["date"] >= cutoff_date]

In [4]:
bruno.loc[:, "date"] = pd.to_datetime(bruno["date"], format="%Y-%m-%d")
# bruno.loc[:, "player_name"] = bruno["player_name"].apply(
#     lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
# )

In [5]:
teg.loc[:, "date"] = pd.to_datetime(teg["date"], format="%Y-%m-%d")

teg.loc[:, "team_1"] = teg["team_1"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)
teg.loc[:, "team_2"] = teg["team_2"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)

In [13]:
# Rating por evento
a = bruno.groupby(["team", "match_id", "event_id"])["rating"].mean()

# Rating por match
a = bruno.groupby(["team", "match_id"])["rating"].mean()

# Rating por time
a = bruno.groupby(["team"])["rating"].mean()

# i = a.iterrows()
# print(next(i))
# apply(
#     print
# )
# a.values.max()
# a.mean()
# for i in a[a >= 1.1].index.unique():
#     print(i)
# bruno[bruno["team"] == "OpTic India"]
a

team
#FREEIBP    1.157778
-0_o-       0.692000
.4sv        0.942000
.Sweden     0.962000
/dab/       1.128000
              ...   
zZzZz       0.906000
zdarova     1.078000
zurditos    0.822000
zuttt       0.982000
zxcv        1.053600
Name: rating, Length: 2991, dtype: float64

In [7]:
teams = frozenset(
    map(
        lambda x: tuple(sorted(x)),
        teg.groupby(["team_1", "team_2"]).size().index.to_numpy()
    )
)

In [8]:
uteams = set(teg["team_1"].unique())
uteams.update(teg["team_2"].unique())

In [9]:
if cutoff_date:
    filename = "teams_" + cutoff_date.replace("-","_") + ".gml"
else:
    filename = "teams.gml"

if not os.path.isdir("gml"):
    os.mkdir("gml")

if not os.path.isfile(f"gml/{filename}"):
    with open(f"gml/{filename}", "w+") as f:
        gml = "graph [\n    directed 0\n"
        for team in uteams:
            gml += "    node [\n"
            gml += f"        id \"{team}\"\n"
            gml += "    ]\n"
        c = 0
        for team in teams:
            gml += "    edge [\n"
            gml += f"        source \"{team[0]}\"\n"
            gml += f"        target \"{team[1]}\"\n"
            gml += "    ]\n"
        gml += "]\n"
        f.write(gml)

In [10]:
g = fm.load(f"gml/{filename}")

In [11]:
g.set_all_nodes(size=10, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.25))

In [ ]:
g.move("kamada_kawai")

In [ ]:
g.draw()

In [17]:
ei = nx.eigenvector_centrality(g)
# helionp.loc[:, "bc"] = pd.DataFrame.from_records(list(bc.items()))[1].values

In [32]:
k = pd.DataFrame.from_records(list(ei.items())).set_index(0)
k["mr"] = a
k = k.dropna()

# k[k.isna().any(axis=1)]
k

,1,mr
0,,
Minttu,0.004230,1.062000
Soberano,0.000214,0.838000
EnVyUs Academy,0.075797,1.107352
Skjuta Hess,0.000007,0.856000
Assassins,0.000021,1.170182
...,...,...
90ss,0.000197,1.084000
Odense,0.006775,0.978690
Revelation,0.000237,0.918000


In [33]:
# x = sm.add_constant(helionp[['con']])
x = k[[1]]
# x = list(map(lambda x: x[0], helionp.index))
y = k["mr"]
model = sm.OLS(y, x) # , missing='drop')
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     mr   R-squared (uncentered):                   0.212
Model:                            OLS   Adj. R-squared (uncentered):              0.211
Method:                 Least Squares   F-statistic:                              316.6
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    6.29e-63
Time:                        05:18:01   Log-Likelihood:                         -1548.6
No. Observations:                1180   AIC:                                      3099.
Df Residuals:                    1179   BIC:                                      3104.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
1             16.0078      0.900     17.794      0.000      14.243      17.773
==============================================================================
Omnibus:                      702.846   Durbin-Watson:                   0.401
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5018.374
Skew:                          -2.793   Prob(JB):                         0.00
Kurtosis:                      11.418   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""